In [2]:
import subprocess
import os
import pandas as pd

def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    """
    Učitava sekvencu iz FASTA formata fajla, uklanja zaglavlje i vraća sekvencu kao string.
    """
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    
    sekvenca = "".join([linija.strip() for linija in linije[1:]])  # Početni red je zaglavlje
    return sekvenca

def pokreni_vsl2_na_fajlu(fajl_putanja):
    """
    Pokreće VSL2 na datom FASTA fajlu i vraća parsirane rezultate.
    """
    vsl2_putanja = "/home/pc/Desktop/VSL2/VSL2.jar"
    rezultati = subprocess.run(
        ['java', '-jar', vsl2_putanja, f'-s:{fajl_putanja}'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    if rezultati.stderr:
        print(f"Greska pri pokretanju VSL2: {rezultati.stderr}")
    
    return parse_vsl2b_output(rezultati.stdout)

def parse_vsl2b_output(vsl2b_output):
    """
    Parsira izlaz iz VSL2B programa i klasifikuje svaku poziciju u sekvenci kao 'uredjenu' ili 'neuredjenu'.
    """
    results = []
    for line in vsl2b_output.splitlines():
        line = line.strip()
        # Preskoči linije koje nisu podaci o sekvenci
        if not line or line.startswith("Prediction Scores") or line.startswith("VSL2") or line.startswith("Center"):
            continue
        parts = line.split()
        if len(parts) >= 3:
            try:
                pos = int(parts[0])  # Pozicija
                amino_acid = parts[1]  # Aminokiselina
                score = float(parts[2])  # VSL2B score
                classification = "uređena" if score <= 0.5 else "neuređena"
                results.append((pos, amino_acid, score, classification))
            except ValueError:
                # Ignoriši linije koje ne mogu da se parsiraju
                print(f"Skipping invalid line: {line}")
                continue
    return results

def generisi_fasta_za_prozor(prozor, indeks):
    """
    Generiše FASTA fajl za svaki prozor, ali bez zaglavlja.
    """
    fasta_putanja = f"Spike glycoprotein/Prozori/prozor_{indeks}.fasta"
    with open(fasta_putanja, 'w') as f:
        f.write(f"{prozor.strip()}")  # Samo sekvenca, bez zaglavlja
    return fasta_putanja


def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=98, max_duzina=100):
    prozori = []
    oznake_prozora = []
    duzine = []
    pozicije = []
    
    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"
            duzine.append(duzina)
            pozicije.append(pozicija)
            
            fasta_putanja = generisi_fasta_za_prozor(podsekvenca, i)
            vsl2_rezultati = pokreni_vsl2_na_fajlu(fasta_putanja)
            
            podsekvenca_oznake = [result[3] for result in vsl2_rezultati]
            
            if podsekvenca_oznake.count("uređena") > len(podsekvenca_oznake) / 2:
                oznake_prozora.append("uređena")
            else:
                oznake_prozora.append("neuređena")
            
            prozori.append(podsekvenca)
    
    return duzine, pozicije, prozori, oznake_prozora

# Učitavanje sekvence
fajl_putanja = "Spike glycoprotein/YP_138523.1 | spike glycoprotein.fasta"
#Nucleoprotein/YP_138520.1 |nucleoprotein.csv
#Membranski protein/YP_138522.1 |matrix protein.csv
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

duzine, pozicije, prozori, oznake_prozora = generisi_prozore_i_klasifikuj(sekvenca)

df_sekvence = pd.DataFrame({
    "Duzina": duzine,
    "Pozicija": pozicije,
    "Sekvenca": prozori,
    "VSL2 oznaka": oznake_prozora
})

print(df_sekvence.head())
df_sekvence.to_csv("Spike glycoprotein/Klasifikovane_sekvence_VSL2_YP_138523.1|spike_glycoprotein_98-100.csv", index=False)


Skipping invalid line: Temple University, Philadelphia, PA
Skipping invalid line: Predicted Disordered Regions:
Skipping invalid line: NO.     RES.    PREDICTION      DISORDER
Skipping invalid line: Temple University, Philadelphia, PA
Skipping invalid line: Predicted Disordered Regions:
Skipping invalid line: NO.     RES.    PREDICTION      DISORDER
Skipping invalid line: Temple University, Philadelphia, PA
Skipping invalid line: Predicted Disordered Regions:
Skipping invalid line: NO.     RES.    PREDICTION      DISORDER
Skipping invalid line: Temple University, Philadelphia, PA
Skipping invalid line: Predicted Disordered Regions:
Skipping invalid line: NO.     RES.    PREDICTION      DISORDER
Skipping invalid line: Temple University, Philadelphia, PA
Skipping invalid line: Predicted Disordered Regions:
Skipping invalid line: NO.     RES.    PREDICTION      DISORDER
Skipping invalid line: Temple University, Philadelphia, PA
Skipping invalid line: Predicted Disordered Regions:
Skipping